In [1]:
import os
import pandas as pd
import random
from sklearn.model_selection import train_test_split

# Paths and configurations
csv_path = "/kaggle/input/newcars/UpdatedCarDataset.csv"  # Replace with your CSV file path
output_dir = "/kaggle/working/yolov11_car"
old_path = "/kaggle/input/car-brand/Public/Public"  # Replace with your base path
images_dir = os.path.join(output_dir, "images")
train_test_split_ratio = 0.8

# Create directories
os.makedirs(output_dir, exist_ok=True)
os.makedirs(images_dir, exist_ok=True)

# Read CSV file without headers
df = pd.read_csv(csv_path, header=None, names=["path", "label"])
df['label'] = df['label'].astype(str)

# Function to copy an image
def copy_image(src, dest):
    try:
        if os.path.exists(src):
            os.makedirs(os.path.dirname(dest), exist_ok=True)
            with open(src, "rb") as fsrc, open(dest, "wb") as fdest:
                fdest.write(fsrc.read())
            return True
    except Exception as e:
        print(f"Failed to copy {src}: {e}")
    return False

# Process each label
for label in df['label'].unique():
    print(f"Processing label: {label}")
    label_df = df[df['label'] == label]
    label_output_dir = os.path.join(images_dir, label)
    os.makedirs(label_output_dir, exist_ok=True)

    # Copy all images for the label
    copied_images = 0
    for _, row in label_df.iterrows():
        relative_path = row['path']
        full_path = os.path.join(old_path, relative_path)  # Construct full path
        output_path = os.path.join(label_output_dir, f"{copied_images}.jpg")
        if copy_image(full_path, output_path):
            copied_images += 1

    print(f"Copied {copied_images} images for label: {label}")

# Split into train and test datasets
for label in os.listdir(images_dir):
    label_path = os.path.join(images_dir, label)
    images = os.listdir(label_path)
    train, test = train_test_split(images, train_size=train_test_split_ratio, random_state=42)

    # Create train/test directories
    train_dir = os.path.join(output_dir, "train", label)
    test_dir = os.path.join(output_dir, "test", label)
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Move files to train/test directories
    for img in train:
        os.rename(os.path.join(label_path, img), os.path.join(train_dir, img))
    for img in test:
        os.rename(os.path.join(label_path, img), os.path.join(test_dir, img))

print("Data preparation completed.")


Processing label: 0
Copied 4393 images for label: 0
Processing label: 1
Copied 2425 images for label: 1
Processing label: 2
Copied 2912 images for label: 2
Processing label: 3
Copied 2648 images for label: 3
Processing label: 4
Copied 6354 images for label: 4
Processing label: 5
Copied 5485 images for label: 5
Processing label: 6
Copied 3384 images for label: 6
Processing label: 7
Copied 2268 images for label: 7
Processing label: 8
Copied 2613 images for label: 8
Data preparation completed.


In [2]:
import os

# Define the path where you want to create the folder
folder_path = '/kaggle/working/yolov11_checkpoints'

# Create the folder (and any missing parent directories)
os.makedirs(folder_path, exist_ok=True)

# Check if the folder has been created
print(f"Folder created at: {folder_path}")

Folder created at: /kaggle/working/yolov11_checkpoints


In [3]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.6/911.6 kB 14.2 MB/s eta 0:00:00a 0:00:01


In [7]:
from ultralytics import YOLO

# Initialize the model
model = YOLO('yolov8n-cls.pt')  # Use YOLOv8 nano classification model
save_path = '/kaggle/working/yolov11_checkpoints'

# Train the model
model.train(data='/kaggle/working/yolov11_car', epochs=30, imgsz=224, batch=32,project=save_path, device = [0,1])

Ultralytics 8.3.65 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/kaggle/working/yolov11_car, epochs=30, time=None, patience=100, batch=32, imgsz=224, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=/kaggle/working/yolov11_checkpoints, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, s